In [47]:
import pandas as pd
import re

# GitHub raw URL for the file
file_url = "https://github.com/gretchenmoulton/Master_Thesis/raw/main/ICIO/2016-2020_SML/2020_SML.csv"

# Load the data directly from GitHub
df = pd.read_csv(file_url)

# Set 'V1' column as index
df.set_index('V1', inplace=True)

# Extract country codes from row and column labels
row_countries = df.index.str.extract(r'(^[A-Z]{3})')[0]
col_countries = df.columns.to_series().str.extract(r'(^[A-Z]{3})')[0]

# Assign country codes to index and columns
df.index = row_countries
df.columns = col_countries

# Group and sum by country (no warning version)
country_trade_matrix = df.groupby(df.index).sum().groupby(axis=1, level=0).sum()

# Clean up axis names
country_trade_matrix.index.name = 'Exporter'
country_trade_matrix.columns.name = 'Importer'

# Done!
print(f"Country-to-country matrix shape: {country_trade_matrix.shape}")
print(country_trade_matrix.head())


Country-to-country matrix shape: (79, 78)
Importer          ARG           AUS          AUT          BEL          BGD  \
Exporter                                                                     
ARG       562824.7507  6.201133e+02      65.0193     841.1857     272.3269   
AUS          251.4600  2.314490e+06     241.0893     683.0254     736.9871   
AUT          156.9267  1.181433e+03  608376.5222    2610.8082     418.9805   
BEL          267.4217  1.011277e+03    2308.2214  774017.8951     470.5185   
BGD           29.3197  9.029133e+02     619.3206     413.7371  531446.8575   

Importer        BGR       BLR        BRA       BRN        CAN  ...        SVN  \
Exporter                                                       ...              
ARG         32.7991  100.3029  8183.0333   13.2685   440.0203  ...    23.2472   
AUS         34.8789   14.8557  1135.7054  340.6986  1972.0348  ...    39.2039   
AUT       1013.6257  176.0735  1008.0837    5.1708  1463.4801  ...  2488.1177   
BEL   

/var/folders/7x/5wqf5crs79v4w9d_tn_v33400000gn/T/ipykernel_46047/3417231139.py:22: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  country_trade_matrix = df.groupby(df.index).sum().groupby(axis=1, level=0).sum()


I am removing OUT which is the sum across each row and TLS which is Timor-Leste and they only import and have no exports which was making the matrix not symmetric (79x78). Final agggregated matrix across industries is 77x77 as thats how many countries we have. Important to note ROW is "Rest of world" and acts like a country in this case. 

In [33]:
# Define countries to drop
drop_countries = ['OUT', 'TLS']

# Drop from both index (exporters) and columns (importers)
country_trade_matrix = country_trade_matrix.drop(index=drop_countries, errors='ignore')
country_trade_matrix = country_trade_matrix.drop(columns=drop_countries, errors='ignore')

# Check result
print(f"Final shape after dropping: {country_trade_matrix.shape}")
print(country_trade_matrix.head())


Final shape after dropping: (77, 77)
Importer          ARG           AUS          AUT          BEL          BGD  \
Exporter                                                                     
ARG       562824.7507  6.201133e+02      65.0193     841.1857     272.3269   
AUS          251.4600  2.314490e+06     241.0893     683.0254     736.9871   
AUT          156.9267  1.181433e+03  608376.5222    2610.8082     418.9805   
BEL          267.4217  1.011277e+03    2308.2214  774017.8951     470.5185   
BGD           29.3197  9.029133e+02     619.3206     413.7371  531446.8575   

Importer        BGR       BLR        BRA       BRN        CAN  ...        SVN  \
Exporter                                                       ...              
ARG         32.7991  100.3029  8183.0333   13.2685   440.0203  ...    23.2472   
AUS         34.8789   14.8557  1135.7054  340.6986  1972.0348  ...    39.2039   
AUT       1013.6257  176.0735  1008.0837    5.1708  1463.4801  ...  2488.1177   
BEL        

In [48]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Calculate trade flow similarity between countries
# Here, we're considering each row in the country_trade_matrix as the trade flows of a country
trade_similarity_matrix = cosine_similarity(country_trade_matrix)

# Convert to DataFrame for better readability
trade_similarity_df = pd.DataFrame(trade_similarity_matrix, index=country_trade_matrix.index, columns=country_trade_matrix.index)
print(trade_similarity_df)

Exporter       ARG       AUS       AUT       BEL       BGD       BGR  \
Exporter                                                               
ARG       1.000000  0.558167  0.591826  0.601023  0.546455  0.597632   
AUS       0.558167  1.000000  0.597626  0.606034  0.551772  0.603093   
AUT       0.591826  0.597626  1.000000  0.646679  0.585643  0.644947   
BEL       0.601023  0.606034  0.646679  1.000000  0.593914  0.653462   
BGD       0.546455  0.551772  0.585643  0.593914  1.000000  0.590964   
...            ...       ...       ...       ...       ...       ...   
TWN       0.595969  0.606745  0.638701  0.647567  0.588753  0.644477   
UKR       0.569261  0.574344  0.610713  0.619277  0.562939  0.617126   
USA       0.544398  0.548186  0.585624  0.599953  0.540104  0.587536   
VNM       0.608650  0.617496  0.650227  0.659854  0.599976  0.656395   
ZAF       0.564278  0.570183  0.604574  0.613677  0.557355  0.610558   

Exporter       BLR       BRA       BRN       CAN  ...       SWE

**K_Means Clustering**

In [49]:
from sklearn.cluster import KMeans

# Number of clusters (you can adjust this number)
num_clusters = 5

# Perform KMeans clustering on the similarity matrix (using the distance)
# We use 1 - cosine similarity to compute the distance matrix
distance_matrix = 1 - trade_similarity_matrix

# Apply KMeans clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
country_clusters = kmeans.fit_predict(distance_matrix)

# Add the clusters as a new column to the DataFrame for easy identification
country_trade_matrix['Cluster'] = country_clusters

# Print out the countries in each cluster
print(f"Countries in each cluster:")
for cluster_num in range(num_clusters):
    cluster_countries = country_trade_matrix[country_trade_matrix['Cluster'] == cluster_num].index.tolist()
    print(f"Cluster {cluster_num + 1}:")
    print(cluster_countries)
    print("\n")


Countries in each cluster:
Cluster 1:
['CAN', 'CHL', 'CMR', 'CRI', 'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'HRV', 'ISL', 'ISR', 'ITA', 'JOR', 'KOR', 'LAO', 'MAR', 'NOR', 'PER', 'PHL', 'PRT', 'ROU', 'ROW', 'RUS', 'SAU', 'SWE', 'TUR', 'UKR', 'ZAF']


Cluster 2:
['OUT']


Cluster 3:
['AUT', 'BEL', 'BGR', 'BLR', 'BRN', 'CHE', 'CYP', 'CZE', 'DEU', 'DNK', 'EST', 'HUN', 'KHM', 'LTU', 'LVA', 'MEX', 'MYS', 'NLD', 'POL', 'SVK', 'SVN', 'THA', 'TUN', 'TWN', 'VNM']


Cluster 4:
['IRL', 'LUX', 'MLT', 'SGP', 'TLS']


Cluster 5:
['ARG', 'AUS', 'BGD', 'BRA', 'CHN', 'CIV', 'COL', 'EGY', 'IDN', 'IND', 'JPN', 'KAZ', 'MMR', 'NGA', 'NZL', 'PAK', 'SEN', 'USA']




Here I use 10 clusters instead of 5 to see how it differs, still using K-means clustering:

In [50]:


from sklearn.cluster import KMeans

# Number of clusters (you can adjust this number)
num_clusters = 10

# Perform KMeans clustering on the similarity matrix (using the distance)
# We use 1 - cosine similarity to compute the distance matrix
distance_matrix = 1 - trade_similarity_matrix

# Apply KMeans clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
country_clusters = kmeans.fit_predict(distance_matrix)

# Add the clusters as a new column to the DataFrame for easy identification
country_trade_matrix['Cluster'] = country_clusters

# Print out the countries in each cluster
print(f"Countries in each cluster:")
for cluster_num in range(num_clusters):
    cluster_countries = country_trade_matrix[country_trade_matrix['Cluster'] == cluster_num].index.tolist()
    print(f"Cluster {cluster_num + 1}:")
    print(cluster_countries)
    print("\n")

Countries in each cluster:
Cluster 1:
['FIN', 'GRC', 'HRV', 'ISL', 'ISR', 'ITA', 'PHL', 'ROU', 'ROW', 'RUS', 'UKR', 'ZAF']


Cluster 2:
['OUT']


Cluster 3:
['IRL', 'LUX', 'MLT', 'SGP', 'TLS']


Cluster 4:
['AUS', 'BGD', 'BRA', 'EGY', 'IDN', 'IND', 'JPN', 'KAZ', 'NZL']


Cluster 5:
['COL', 'NGA', 'PAK']


Cluster 6:
['CHL', 'HKG', 'JOR', 'KOR', 'LAO', 'MYS', 'NOR', 'SAU', 'SWE']


Cluster 7:
['ARG', 'CIV', 'CMR', 'ESP', 'FRA', 'GBR', 'MAR', 'MMR', 'PER', 'PRT', 'SEN', 'TUR']


Cluster 8:
['CAN', 'CRI', 'MEX']


Cluster 9:
['AUT', 'BEL', 'BGR', 'BLR', 'BRN', 'CHE', 'CYP', 'CZE', 'DEU', 'DNK', 'EST', 'HUN', 'KHM', 'LTU', 'LVA', 'NLD', 'POL', 'SVK', 'SVN', 'THA', 'TUN', 'TWN', 'VNM']


Cluster 10:
['CHN', 'USA']




Here we use spectral clustering, it ends up creating 5 clusters but they differ from the first 5 created. Below I have included an analysis on how they differ

**Spectral Clustering**

In [51]:
import numpy as np

distance_matrix = 1 - trade_similarity_matrix
print("Min distance:", distance_matrix.min().min())
print("Max distance:", distance_matrix.max().max())
print("Mean distance:", distance_matrix.mean().mean())


Min distance: -8.881784197001252e-16
Max distance: 0.9863125335513104
Mean distance: 0.3946422541080441


In [52]:
from sklearn.cluster import SpectralClustering

# Spectral clustering uses similarity directly
spectral = SpectralClustering(
    n_clusters=5,
    affinity='precomputed',
    random_state=42
)

clusters = spectral.fit_predict(trade_similarity_matrix)
country_trade_matrix['Cluster'] = clusters

# Display countries in each cluster
print("Countries in each cluster:\n")
for cluster_id in sorted(country_trade_matrix['Cluster'].unique()):
    countries = country_trade_matrix[country_trade_matrix['Cluster'] == cluster_id].index.tolist()
    print(f"Cluster {cluster_id + 1}:")
    print(countries)
    print()



Countries in each cluster:

Cluster 1:
['ARG', 'AUS', 'BGD', 'BLR', 'BRA', 'BRN', 'CHL', 'CHN', 'CIV', 'CMR', 'COL', 'CRI', 'EGY', 'GRC', 'HKG', 'IDN', 'IND', 'ISR', 'JOR', 'JPN', 'KAZ', 'KHM', 'KOR', 'LAO', 'MMR', 'MYS', 'NGA', 'NZL', 'PAK', 'PER', 'PHL', 'RUS', 'SAU', 'SEN', 'SGP', 'THA', 'TUR', 'TWN', 'UKR', 'VNM', 'ZAF']

Cluster 2:
['OUT']

Cluster 3:
['AUT', 'BEL', 'BGR', 'CHE', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'HRV', 'HUN', 'IRL', 'ISL', 'ITA', 'LTU', 'LVA', 'MAR', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'SWE', 'TLS', 'TUN']

Cluster 4:
['CYP', 'LUX', 'MLT', 'ROW']

Cluster 5:
['CAN', 'MEX', 'USA']



**SUMMARY of the two groups of 5 clusters**

K Means Cluster 1: Côte d'Ivoire, Cameroon, Hong Kong, Indonesia, India, Iceland, Luxembourg, Malta, Myanmar, Malaysia, Philippines, **Rest of the World**, Saudi Arabia, Senegal, Thailand, Taiwan, South Africa

K Means Cluster 2: Bangladesh, Belarus, Cyprus, Egypt, Estonia, Greece, Kazakhstan, Lithuania, Latvia, Morocco, Nigeria, Pakistan, Russia, Turkey, Ukraine

K Means Cluster 3: Argentina, Australia, Brazil, Brunei, Chile, **China**, Colombia, Japan, South Korea, Laos, New Zealand, Peru

K Means Cluster 4: Austria, Belgium, Bulgaria, Czech Republic, Germany, Spain, Finland, France, United Kingdom, Croatia, Hungary, Italy, Netherlands, Norway, Poland, Portugal, Romania, Slovakia, Slovenia, Sweden, Tunisia

K Means Cluster 5: Canada, Switzerland, Costa Rica, Denmark, Ireland, Israel, Jordan, Cambodia, Mexico, Singapore, **United States**, Vietnam



Spectral Cluster 1: Belarus, Kazakhstan, Russia, Ukraine

Spectral Cluster 2: Australia, Brunei, Chile, **China**, Hong Kong, Indonesia, India, Japan, South Korea, Laos, Myanmar, Malaysia, New Zealand, Peru, Philippines, Singapore, Thailand, Taiwan, Vietnam

Spectral  Cluster 3: Denmark, Estonia, Finland, Lithuania, Latvia, Norway, Sweden

Spectral Cluster 4: Argentina, Bangladesh, Brazil, Canada, Ivory Coast (Côte d'Ivoire), Cameroon, Colombia, Costa Rica, Cyprus, Egypt, United Kingdom, Greece, Ireland, Iceland, Israel, Jordan, Cambodia, Mexico, Malta, Nigeria, Pakistan, **Rest of World (ROW)**, Saudi Arabia, Senegal, **United States**, South Africa

Spectral Cluster 5: Austria, Belgium, Bulgaria, Switzerland, Czech Republic, Germany, Spain, France, Croatia, Hungary, Italy, Luxembourg, Morocco, Netherlands, Poland, Portugal, Romania, Slovakia, Slovenia, Tunisia, Turkey


ChatGPT summary of the changes but its a bit confusing

**European Cluster:**
First Set: Austria, Belgium, Bulgaria, Czech Republic, Germany, Spain, Finland, France, United Kingdom, Croatia, Hungary, Italy, Netherlands, Norway, Poland, Portugal, Romania, Slovakia, Slovenia, Sweden, Tunisia.

Second Set: Austria, Belgium, Bulgaria, Switzerland, Czech Republic, Germany, Spain, France, Croatia, Hungary, Italy, Luxembourg, Morocco, Netherlands, Poland, Portugal, Romania, Slovakia, Slovenia, Tunisia, Turkey.

Changes:

Added: Luxembourg, Morocco, Turkey, Switzerland.

Left: None.

**Asia-Pacific Cluster:**
First Set: Bangladesh, Belarus, Cyprus, Egypt, Estonia, Greece, Kazakhstan, Lithuania, Latvia, Morocco, Nigeria, Pakistan, Russia, Turkey, Ukraine.

Second Set: Australia, Brunei, Chile, **China**, Hong Kong, Indonesia, India, Japan, South Korea, Laos, Myanmar, Malaysia, New Zealand, Peru, Philippines, Singapore, Thailand, Taiwan, Vietnam.

Changes:

Added: Australia, Brunei, Chile, **China**, Hong Kong, Indonesia, India, Japan, South Korea, Laos, Myanmar, Malaysia, New Zealand, Peru, Philippines, Singapore, Thailand, Taiwan, Vietnam.

Left: Bangladesh, Belarus, Cyprus, Egypt, Estonia, Greece, Kazakhstan, Lithuania, Latvia, Morocco, Nigeria, Pakistan, Russia, Turkey, Ukraine.

**African & Middle Eastern Cluster:**
First Set: Côte d'Ivoire (Ivory Coast), Cameroon, Hong Kong, Indonesia, India, Iceland, Luxembourg, Malta, Myanmar, Malaysia, Philippines, Rest of the World, Saudi Arabia, Senegal, Thailand, Taiwan, South Africa.

Second Set: Argentina, Bangladesh, Brazil, Canada, Ivory Coast (Côte d'Ivoire), Cameroon, Colombia, Costa Rica, Cyprus, Egypt, United Kingdom, Greece, Ireland, Iceland, Israel, Jordan, Cambodia, Mexico, Malta, Nigeria, Pakistan, **Rest of World (ROW)**, Saudi Arabia, Senegal, **United States**, South Africa.

Changes:

Added: Argentina, Brazil, Canada, Colombia, Costa Rica, Cyprus, Egypt, United Kingdom, Greece, Ireland, Israel, Jordan, Cambodia, Mexico, Nigeria, United States.

Left: Hong Kong, Indonesia, Malta, Myanmar, Philippines, Thailand, Taiwan.

**North American & Western Countries:**
First Set: Canada, Switzerland, Costa Rica, Denmark, Ireland, Israel, Jordan, Cambodia, Mexico, Singapore, **United States**, Vietnam.

Second Set: Austria, Belgium, Bulgaria, Switzerland, Czech Republic, Germany, Spain, France, Croatia, Hungary, Italy, Luxembourg, Morocco, Netherlands, Poland, Portugal, Romania, Slovakia, Slovenia, Tunisia, Turkey.

Changes:

Added: Switzerland stayed, but Canada, Costa Rica, Denmark, Ireland, Israel, Jordan, Cambodia, Mexico, Singapore, United States, Vietnam left.

Left: Canada, United States, Switzerland moved into the European group.

